# P2TR Key Path and Script Path

- The following material adapted from the Bitcoin Optech [Schnorr Taproot workshop](https://bitcoinops.org/en/schorr-taproot-workshop/).

In this section we'll create a P2TR output that commits to a taptree, then explore how we'd spend from it using the key path and script path.

## Prerequisite knowledge
- For all notebooks:
    - A high level understanding of the bitcoin. e.g. [Mastering Bitcoin](https://github.com/bitcoinbook/bitcoinbook) by Andreas Antonopoulos UTXO model, in particular [Chapter 6](https://github.com/bitcoinbook/bitcoinbook/blob/develop/ch06.asciidoc).
    - A conceptual understanding of [hash functions](https://www.thesslstore.com/blog/what-is-a-hash-function-in-cryptography-a-beginners-guide).
    - [Hexadecimal notation](https://inst.eecs.berkeley.edu/~cs61bl/r//cur/bits/decimal-binary-hex.html?topic=lab28.topic&step=2&course=) and [endianness](https://www.freecodecamp.org/news/what-is-endianness-big-endian-vs-little-endian/).


- Specific to this notebook:
    - SHA256, HASH256, HASH160 - '[Hash Functions chapter](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/appendix/hash-functions.ipynb)'
    - Base58 addresses - '[Addresses chapter](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/appendix/addresses.ipynb)'
    - Bitcoin Script basics - '[Bitcoin Script chapter](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/appendix/bitcoin-script.ipynb)'
    - TestShell setup - '[P2PKH chapter](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/chapter1-legacy/p2pkh.ipynb)'
- Specific for taproot:
    - [Elliptic Curve Math Review](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/appendix/elliptic_curve_math_review.ipynb)
    - [Schnorr signatures and TapTweaks](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/chapter3-taproot/schnorr-sig-and-taptweak.ipynb)
    - [TapTrees](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/chapter3-taproot/taproot-taptree.ipynb)
    - [Signature hash evolution](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/appendix/sighash_evolution.ipynb)

## Setup 

### Requirements
For this exercise we'll need Bitcoin Core. This notebook has been tested with [v24.0.1](https://github.com/bitcoin/bitcoin/releases/tag/v24.0.1).

Below, set the paths for:
1. The bitcoin core functional test framework directory.
2. The directory containing bitcoin-tx-tutorial.

**You'll need to edit these next two lines for your local setup.**

In [1]:
path_to_bitcoin_functional_test = "/Users/dariuscognac/bitcoin/test/functional"
path_to_bitcoin_tx_tutorial = "/Users/dariuscognac/Documents/Github/bitcoin-tx-tutorial"

import sys

# Add the functional test framework to our PATH
sys.path.insert(0, path_to_bitcoin_functional_test)
from test_framework.test_shell import TestShell

# Add the bitcoin-tx-tutorial functions to our PATH
sys.path.insert(0, path_to_bitcoin_tx_tutorial)
from functions import *
from functions.bip_0340_reference import *

import json

## Scenario

Patricia is a mother of three kids, Adam, Ben, Carla, all in their 20s attending college. She's planning to go backpacking in the countryside where she won't have access to the internet. She decides she wants to put away some funds that her kids could use in an emergency if any of them ever needed money while she was away. She's happy to let any of the kids spend the funds under any condition. Ideally she wants her transaction to be as private as possible.

Q: What options does she have?

She could simply give all three of her kids the private key to a utxo/wallet, but she knows it's not the best security practice to distribute secret information like that. She also wants it to be provable who spent the in the case it does get spent. Before taproot, she probably would have been best off creating a P2WSH transaction where the script allowed any of the children's private keys to spend the output. Now that taproot transactions are supported, she can create an address that commits to a taptree, where each leaf of the taptree commits to each of the children's public keys.

Compared to P2WSH, this transaction will be more private. Patricia can retain the ability to spend the funds using the internal public key and never reveal that it committed to a script. Alternatively, if one of the children spends the output, only their public key gets revealed. 

## Part 1: Constructing a P2TR output that commits to a taptree

In order to create this output, we'll need to perform the following steps. For more on taptrees see the chapter on
[TapTrees](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/taproot_taptree.ipynb):
1. Compute TapLeaves A, B and C (for Adam, Ben, Carla).
2. Compute Internal TapBranch AB.
3. Compute TapTweak
4. Derive the bech32m address.

In [2]:
TAPSCRIPT_VER = bytes([0xc0])
internal_privkey = bytes.fromhex("83a5f1039118fbb4276cac2db41d236c1c1790d97d955c228fa3bde439fbec2a")
internal_pubkey = tr.pubkey_gen(internal_privkey)

# Derive pay-to-pubkey scripts
privkeyA = bytes.fromhex("1059bf26660804ced9a3286a16497d7e70692d14dc04e1220c2dbef3667b74f7")
pubkeyA = tr.pubkey_gen(privkeyA)
privkeyB = bytes.fromhex("2b22bf11ab862a35f16301c0afc7afe60f66d31fc29645f79c2ab43655e65d33")
pubkeyB = tr.pubkey_gen(privkeyB)
privkeyC = bytes.fromhex("7f8b28e51da049bf63e31d3a3261579c0f5c1fc8058c65a79482814e5061f9f6")
pubkeyC = tr.pubkey_gen(privkeyC)

# Pay to pubkey scripts
scriptA = b"\x20" + pubkeyA + b"\xac"
scriptB = b"\x20" + pubkeyB + b"\xac"
scriptC = b"\x20" + pubkeyC + b"\xac"

# Method: Returns tapbranch hash. Child hashes are lexographically sorted and then concatenated.
# l: tagged hash of left child
# r: tagged hash of right child
def tapbranch_hash(l, r):
    return tagged_hash("TapBranch", b''.join(sorted([l,r])))

# 1) Compute TapLeaves A, B and C.
# Method: pushbytes(data) is a function which adds compactsize to input data.
hash_inputA =  TAPSCRIPT_VER + pushbytes(scriptA)
hash_inputB =  TAPSCRIPT_VER + pushbytes(scriptB)
hash_inputC =  TAPSCRIPT_VER + pushbytes(scriptC)
taggedhash_leafA =  tagged_hash("TapLeaf", hash_inputA)
taggedhash_leafB =  tagged_hash("TapLeaf", hash_inputB)
taggedhash_leafC =  tagged_hash("TapLeaf", hash_inputC)

# 2) Compute Internal node TapBranch AB.
# Method: use tapbranch_hash() function
internal_nodeAB = tapbranch_hash(taggedhash_leafA, taggedhash_leafB)

# 3) Compute TapTweak.
rootABC =  tapbranch_hash(internal_nodeAB, taggedhash_leafC)
taptweak =  tagged_hash("TapTweak", internal_pubkey + rootABC)

# 4) Derive the bech32m address.
negated, taproot_pubkey = taproot_tweak_pubkey(internal_pubkey, rootABC)
print("TapTweak:", taptweak.hex())
spk = bytes.fromhex("5120") + taproot_pubkey
bech32m_address = spk_to_bech32(spk, 'regtest')
print('Bech32m address:', bech32m_address)

TapTweak: 8e53fc0da2e9e8e27404703010ae519e85576ef6c73dde1b2bffac7f17b114a9
Bech32m address: bcrt1p5jhcyymfj7tkgv0jca4pz7tx9uzvznu0mlfymey6ph63f9h8x0gs7683vc


Now we are ready to set up the bitcoin `TestShell` and fund this address. For more on these steps you can view the [function definitions](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/functions/setup_testshell.py) or look at the first [P2PKH notebook](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/chapter1-legacy/p2pkh.ipynb) example.

In [3]:
node = setup_testshell()
txid_to_spend, index_to_spend = fund_address(node, bech32m_address, 2.001)
print(f"txid: {txid_to_spend}, {index_to_spend}")

2023-01-01T09:42:22.576000Z TestFramework (INFO): Initializing test directory /var/folders/r5/yk8yg2xs1gs8xzkn5l8vr72w0000gn/T/bitcoin_func_test_a30mhq8s
txid: 5e61d25df2ca61d6be58b98a434e3ddd7d70dabd5a0c38bff8c97df40c02420f, 1


## Part 2: Spending along the Key Path

### Scenario
Patricia comes back from her backpacking trip and thankfully none of her children needed to use the emergency funds she allocated for them. She decides to move majority of the funds (1.5 btc) into her cold wallet, and the rest (0.5 btc) to her hot wallet using the internal public key via a key path spend.

The first step she'll need to do is derive the tweaked internal private key. For more on this, review the section on [Schnorr signatures and TapTweaks](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/schnorr_sig_and_taptweak.ipynb).

In [4]:
taproot_privkey = taproot_tweak_seckey(internal_privkey, rootABC)

Next, she'll need to construct the unsigned transaction to 

In [5]:
# Set the output scriptPubkeys and amounts
output1_value_sat = int(float("1.5") * 100000000)
output1_spk = bytes.fromhex("0014fc7250a211deddc70ee5a2738de5f07817351cef")
output2_value_sat = int(float("0.5") * 100000000)
output2_spk = bytes.fromhex("0014531260aa2a199e228c537dfa42c82bea2c7c1f4d")
output2_spk.hex()

'0014531260aa2a199e228c537dfa42c82bea2c7c1f4d'

In [6]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER (new to segwit)
marker = bytes.fromhex("00")

# FLAG (new to segwit)
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
txid = (bytes.fromhex(txid_to_spend))[::-1]
index = index_to_spend.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# use 0xffffffff unless you are using OP_CHECKSEQUENCEVERIFY, locktime, or rbf
sequence = bytes.fromhex("ffff ffff")

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

# OUTPUTS
# 0x02 for out two outputs
output_count = bytes.fromhex("02")

# OUTPUT 1 
output1_value = output1_value_sat.to_bytes(8, byteorder="little", signed=True)
# 'output1_spk' already defined at the start of the script

# OUTPUT 2
output2_value = output2_value_sat.to_bytes(8, byteorder="little", signed=True)
# 'output2_spk' already defined at the start of the script

outputs = (
    output1_value
    + varint_len(output1_spk)
    + output1_spk
    + output2_value
    + varint_len(output2_spk)
    + output2_spk
)

# LOCKTIME
locktime = bytes.fromhex("0000 0000")

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)
print("unsigned_tx: ", unsigned_tx.hex())

unsigned_tx:  02000000010f42020cf47dc9f8bf380c5abdda707ddd3d4e438ab958bed661caf25dd2615e0100000000ffffffff0280d1f00800000000160014fc7250a211deddc70ee5a2738de5f07817351cef80f0fa0200000000160014531260aa2a199e228c537dfa42c82bea2c7c1f4d00000000


Now she's ready to compute the sighash and sign it with a schnorr signature using her tweaked internal key. For more on this section see [Signature hash evolution](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/sighash_evolution.ipynb).

In [7]:
sighash_epoch = bytes.fromhex("00")
index_of_this_input = bytes.fromhex("0000 0000")

# Control
hash_type = bytes.fromhex("00") # SIGHASH_DEFAULT (a new sighash type meaning implied SIGHASH_ALL)

# Transaction data
sha_prevouts = sha256(txid + index)

input_amount_sat = int(2.001 * 100_000_000)
input_amounts = input_amount_sat.to_bytes(8, byteorder="little", signed=False)
sha_amounts = sha256(input_amounts)

sha_scriptpubkeys = sha256(
    varint_len(spk)
    + spk
)

sha_sequences = sha256(sequence)

sha_outputs = sha256(outputs) ######

# Data about this input
spend_type = bytes.fromhex("00") # no annex present

sig_msg = (
    sighash_epoch
    + hash_type
    + version
    + locktime
    + sha_prevouts
    + sha_amounts
    + sha_scriptpubkeys
    + sha_sequences
    + sha_outputs
    + spend_type
    + index_of_this_input
)

tag_hash = sha256("TapSighash".encode())
sighash = sha256(tag_hash + tag_hash + sig_msg)

Now with the sighash she can sign it with her tweaked internal key. For more on schnorr signatures see [Schnorr signatures and TapTweaks](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/schnorr_sig_and_taptweak.ipynb).

In [8]:
# Sign the transaction
aux_rand = bytes.fromhex("0000000000000000000000000000000000000000000000000000000000000000")
signature = tr.schnorr_sign(sighash, taproot_privkey, aux_rand)
# Sighash flag is not appended for SIGHASH_DEFAULT

Finally the last step is to add the signature to the witness field and she has the signed transaction!

In [9]:
witness = (
    bytes.fromhex("01") # one stack item in the witness
    + varint_len(signature)
    + signature
)
print(witness.hex())

# the final signed transaction
signed_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs
    + output_count
    + outputs
    + witness
    + locktime
)

print("signed transaction: ",signed_tx.hex())

0140ae7803efebf0bd7bfdf5a3d7cdd18a77190d08288a72f6bc28261be426cfd7b65055f6f324e0cc7b3e62c03ad4eaf301cc7851bde78031f759e1abaec9921764
signed transaction:  020000000001010f42020cf47dc9f8bf380c5abdda707ddd3d4e438ab958bed661caf25dd2615e0100000000ffffffff0280d1f00800000000160014fc7250a211deddc70ee5a2738de5f07817351cef80f0fa0200000000160014531260aa2a199e228c537dfa42c82bea2c7c1f4d0140ae7803efebf0bd7bfdf5a3d7cdd18a77190d08288a72f6bc28261be426cfd7b65055f6f324e0cc7b3e62c03ad4eaf301cc7851bde78031f759e1abaec992176400000000


In [10]:
result = node.testmempoolaccept(rawtxs=[signed_tx.hex()])
print(result)

[{'txid': '92a4387f65646140ccb75a611874d58defb94a3a7bb46b0ba4d59ef87e29aec9', 'wtxid': 'bdeeda66b862d30a3dffecc9fad27c46490bc677b96e550c265ff7e1101830ef', 'allowed': True, 'vsize': 130, 'fees': {'base': Decimal('0.00100000')}}]


Note that unlike P2SH or P2WSH, the taptree is never revealed.

## Part 3: Spending along the Script Path

### Scenario
While Patricia is out of town, Adam has an emergency medical expense and needs to use some of the bitcoin Patricia set aside for them. He decides to spend the UTXO to fund his hospital bill costing 1.5 btc.

For convenience we'll reuse the same outputs as in part 2. Note that since Adam is spending via script path, he'll need to set the `spend_type` to `02`.

In [11]:
spend_type = bytes.fromhex("02")

The rest of the sighash is very similar to the key path spend, however we also need to append the _common signature message extension_ defined in [BIP 342](https://github.com/bitcoin/bips/blob/master/bip-0342.mediawiki#common-signature-message-extension). Pieter Wuille describes the purpose of these fields in a stack exchange answer [here](https://bitcoin.stackexchange.com/questions/115695/what-are-the-last-bytes-for-in-a-taproot-script-path-sighash/115699#115699).

In [12]:
sighash_epoch = bytes.fromhex("00")
index_of_this_input = bytes.fromhex("0000 0000")

# Control
hash_type = bytes.fromhex("00") # SIGHASH_DEFAULT (a new sighash type meaning implied SIGHASH_ALL)

# Transaction data
sha_prevouts = sha256(txid + index)

input_amount_sat = int(2.001 * 100_000_000)
input_amounts = input_amount_sat.to_bytes(8, byteorder="little", signed=False)
sha_amounts = sha256(input_amounts)
sha_scriptpubkeys = sha256(
    varint_len(spk)
    + spk
)
sha_sequences = sha256(sequence)
sha_outputs = sha256(outputs)

# these next three variables make up the 'common signature message extension'
tapleaf_hash = tagged_hash("TapLeaf", TAPSCRIPT_VER + pushbytes(scriptA))
key_version = bytes.fromhex("00")
code_separator_pos = bytes.fromhex("ffffffff")

sig_msg = (
    sighash_epoch
    + hash_type
    + version
    + locktime
    + sha_prevouts
    + sha_amounts
    + sha_scriptpubkeys
    + sha_sequences
    + sha_outputs
    + spend_type # spend_type = 0x02 for script path
    + index_of_this_input
    # 'common signature message extension' for script path
    + tapleaf_hash
    + key_version
    + code_separator_pos
)

tag_hash = sha256("TapSighash".encode())
sighash = sha256(tag_hash + tag_hash + sig_msg)
print(sighash.hex())

e9877ad0b9a7a935c5c8e9cb819afa130f26036e5623388c895cb45b55c3e7f6


Now Adam is ready to sign it with his private key (`privkeyA`).

In [13]:
aux_rand = bytes.fromhex("0000000000000000000000000000000000000000000000000000000000000000")
signatureA = tr.schnorr_sign(sighash, privkeyA, aux_rand)
# Sighash flag is not appended for SIGHASH_DEFAULT

### Witness elements for taproot script path spend

The witness field for a key path spend transaction is a little more involved than previous transaction types. The witness elements should contain the following pattern:

* Witness to spend TapScript_A:

    * `[Stack element(s) satisfying TapScript_A]`
    * `[TapScript_A]` 
    * `[Controlblock c]`

Compared to the script spend path of a taproot with a single committed tapscript, the controlblock spending a taproot containing multiple tapscripts will also include a script inclusion proof.

* Controlblock c contains:

    * `[Tapscript Version]` 
        * `0xfe & c[0]`
    * `[Parity bit (oddness of Q's y-coordinate)]`
        * `0x01 & c[0]` 
    * `[Internal Public Key]` 
        * `c[1:33]`
    * `[Script Inclusion Proof]` 
        * `n x 32Bytes`
        
Note that this script inclusion proof is a 32B multiple and its size will depend on the position of tapscript in the taptree structure.

In [14]:
control_byte = (TAPSCRIPT_VER[0] | negated).to_bytes(1, "big")

control_block = (
    control_byte
    + internal_pubkey
    + taggedhash_leafB
    + taggedhash_leafC
    )

Now we can construct the witness field with 3 elements (each proceeded with the variable length integer), and complete the transaction!

In [15]:
witness = (
    bytes.fromhex("03") 
    + varint_len(signatureA)
    + signatureA
    + varint_len(scriptA)
    + scriptA
    + varint_len(control_block)
    + control_block
)

# the final signed transaction
signed_script_path_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs
    + output_count
    + outputs
    + witness
    + locktime
)

print("signed transaction: ",signed_script_path_tx.hex())

signed transaction:  020000000001010f42020cf47dc9f8bf380c5abdda707ddd3d4e438ab958bed661caf25dd2615e0100000000ffffffff0280d1f00800000000160014fc7250a211deddc70ee5a2738de5f07817351cef80f0fa0200000000160014531260aa2a199e228c537dfa42c82bea2c7c1f4d03408624784388d931c6155db7f44ef7a503a10e36cea2c2354cdbdbf8b8d6ade9d3ef2e5f173299768e56d66bdb1ccc2d32eebf1cabfb435a1001343425219fb42c2220731bbf2e7163d87b12d66d3795655790691e59802fd1c578c2e06287555e3c28ac61c1031845925dcca99bc5689ce422b9204ff9721de8416b984b8a6b930e30352225a1494f24cfdfc93532412675eece318c63414dd02b8750947177b141e082076e2b71b71e05a8cd9771485b9248025e46871caa6585164a1fdf9e634d61023fd800000000


In [16]:
result = node.testmempoolaccept(rawtxs=[signed_tx.hex()])
print(result)

[{'txid': '92a4387f65646140ccb75a611874d58defb94a3a7bb46b0ba4d59ef87e29aec9', 'wtxid': 'bdeeda66b862d30a3dffecc9fad27c46490bc677b96e550c265ff7e1101830ef', 'allowed': True, 'vsize': 130, 'fees': {'base': Decimal('0.00100000')}}]


## Quiz

1. How are the signature opcodes`OP_CHECKSIG` and `OP_CHECKSIGVERIFY` treated in taproot inputs compared to non-taproot inputs.
2. What are tagged hashed, and what is the advantage of using them compared to regular hashes?
3. In the case of a key-path spend, is it possible to tell from the output whether the public key had been tweaked to 

 ## Answers

1. For taproot inputs, the opcodes`OP_CHECKSIG` and `OP_CHECKSIGVERIFY` are used to verify Schnorr signatures rather than ECDSA signatures.
2. A tagged hash is a hash that includes an additional piece of information called a "tag" in addition to the regular data being hashed. An advantage of using tagged hashes is that it ensures the hash can only be used in the way it was intended by the creator. For example, in a tap tree, the "TapLeaf" and "TapBranch" tags prevent a hash for a branch being interpreted as a leaf.
3. No. It is impossible to tell from a key-path spend transaction whether the public key had been tweaked or not.

## Exercise

Try replacing Carla's script `scriptC` with a script of a 2-of-2 multisig. Note that taproot uses a new opcode for multisig, `CHECKSIGADD` [BIP342](https://github.com/bitcoin/bips/blob/master/bip-0342.mediawiki#rules-for-signature-opcodes). This [Stack Exchange Answer](https://bitcoin.stackexchange.com/questions/114465/how-does-the-checksigadd-opcode-work-how-does-it-compare-to-its-predecessor-che) may be helpful.